In [ ]:
import set_path

import numpy as np
import laspy

import open3d as o3d

from upcp.utils import ahn_utils
from upcp.utils.interpolation import FastGridInterpolator

from gvl.helper_functions import color_clusters
from gvl.tree_detector import DetectorTree

In [ ]:
tilecode = '2463_9714'

las_file = f'../datasets/ahn_{tilecode}.laz'
ahn_npz_folder = '../../../datasets/Accessibility_oost/ahn/ahn4_npz/'

# AHN classification
AHN_OTHER = 1
AHN_GROUND = 2
AHN_BUILDING = 6
AHN_WATER = 9
AHN_ARTIFACT = 26

In [ ]:
ahn_reader = ahn_utils.NPZReader(ahn_npz_folder, caching=False)

In [ ]:
las = laspy.read(las_file)

In [ ]:
points_dtype = np.dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'),
                         ('NZ', '<f8'), ('HAG', '<f8'),
                         ('Classification', '<i4'), ('NumberOfReturns', '<i4'), ('ReturnNumber', '<i4')])
points = np.empty(shape=(las.header.point_count,), dtype=points_dtype)

In [ ]:
o3d_pc = o3d.geometry.PointCloud()
o3d_pc.points = o3d.utility.Vector3dVector(las.xyz)
o3d_pc.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                                                      max_nn=30))

In [ ]:
gnd_tile = ahn_reader.filter_tile(tilecode)

# gnd_tile = ahn_utils.fill_gaps(gnd_tile, max_gap_size=10000)

fast_z = FastGridInterpolator(
            gnd_tile['x'], gnd_tile['y'], gnd_tile['ground_surface'])
gnd_z = fast_z(las.xyz)

In [ ]:
points['X'] = las.x
points['Y'] = las.y
points['Z'] = las.z
points['NZ'] = np.asarray(o3d_pc.normals)[:,2]
points['HAG'] = las.z - gnd_z
points['Classification'] = las.classification
points['NumberOfReturns'] = las.number_of_returns
points['ReturnNumber'] = las.return_number

In [ ]:
tree = DetectorTree(points)

In [ ]:
# first clustering step
tree.hdbscan_on_points(min_cluster_size=30, min_samples=10, xyz=False)
tree.convex_hullify(points=tree.clustered_points)
# df_to_pg(tree.tree_df, schema='bomen', table_name='xy_bomen')

In [ ]:
tree.tree_df.plot(column='xy_clusterID', cmap='tab20')

In [ ]:
# second cluster step
tree.find_points_in_polygons(tree.tree_df)
tree.kmean_cluster(tree.xy_grouped_points, round_val=2)
tree.convex_hullify(tree.kmean_grouped_points, kmean_pols=True)
# df_to_pg(tree.tree_df, schema='bomen', table_name='km_bomen')

In [ ]:
tree.tree_df.plot(column='xy_clusterID', cmap='tab20')

In [ ]:
data = tree.kmean_grouped_points.to_records()

In [ ]:
header = laspy.LasHeader(point_format=3, version="1.2")
header.offsets = las.header.offsets
header.scales = las.header.scales

new_las = laspy.LasData(header)

new_las.x = data['X']
new_las.y = data['Y']
new_las.z = data['Z']
new_las.classification = data['Classification']
# new_las.red = data['Red']
# new_las.green = data['Green']
# new_las.blue = data['Blue']

In [ ]:
new_las.write(f'trees_{tilecode}.laz')